#  Метрики качества классификации

#### Вы научитесь:
- вычислять различные меры качества классификации: долю правильных ответов, точность, полноту, AUC-ROC и т.д.
- сравнивать алгоритмы классификации при наличии ограничений на точность или полноту

#### Введение

В задачах классификации может быть много особенностей, влияющих на подсчет качества: различные цены ошибок, несбалансированность классов и т.д. Из-за этого существует большое количество метрик качества — каждая из них рассчитана на определенное сочетание свойств задачи и требований к ее решению.

Меры качества классификации можно разбить на две большие группы: предназначенные для алгоритмов, выдающих номера классов, и для алгоритмов, выдающих оценки принадлежности к классам. К первой группе относятся доля правильных ответов, точность, полнота, F-мера. Ко второй — площади под ROC- или PR-кривой.

#### Реализация в sklearn

Различные метрики качества реализованы в пакете sklearn.metrics.

In [51]:
import pandas as pd
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve

In [36]:
def answer(filename='3-3.txt', *answer):
    file = open(filename, 'w')
    ans = ''
    for i in answer:
        ans += str(i) + ' '
    ans = ans[:-1]
    file.write(ans)
    file.close()

def show(*names):
    for n in names:
        print(n + ': ' + str(eval(n)))

# 1
Загрузите файл classification.csv. В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка pred).

In [28]:
data = pd.read_csv('classification.csv')

print(data.keys())
print(data[:5])

Index(['true', 'pred'], dtype='object')
   true  pred
0     1     0
1     1     1
2     1     1
3     0     0
4     1     1


# 2
Заполните таблицу ошибок классификации. Для этого подсчитайте величины TP, FP, FN и TN согласно их определениям. Например, FP — это количество объектов, имеющих класс 0, но отнесенных алгоритмом к классу 1. Ответ в данном вопросе — четыре числа через пробел.

In [33]:
TP, FP, FN, TN = (0,) * 4

for _, row in data.iterrows():
    true = row['true']
    predict = row['pred']
    
    if true == 1 and predict == 1:
        TP += 1
    elif true == 0 and predict == 1:
        FP += 1
    elif true == 1 and predict == 0:
        FN += 1
    else:
        TN += 1

show('TP', 'FP', 'FN', 'TN')

TP: 43
FP: 34
FN: 59
TN: 64


In [34]:
TP = data[(data.pred == 1) & (data.true == 1)].count().get(0)
TN = data[(data.pred == 0) & (data.true == 0)].count().get(0)
FP = data[(data.pred == 1) & (data.true == 0)].count().get(0)
FN = data[(data.pred == 0) & (data.true == 1)].count().get(0)
show('TP', 'FP', 'FN', 'TN')
answer('3-3-1.txt', TP, FP, FN, TN)

TP: 43
FP: 34
FN: 59
TN: 64


# 3
Посчитайте основные метрики качества классификатора:

- Accuracy (доля верно угаданных) — sklearn.metrics.accuracy_score
- Precision (точность) — sklearn.metrics.precision_score
- Recall (полнота) — sklearn.metrics.recall_score
- F-мера — sklearn.metrics.f1_score

В качестве ответа укажите эти четыре числа через пробел.

In [40]:
accuracy = accuracy_score(y_true=data.true, y_pred=data.pred)
precision = precision_score(data.true, data.pred)
recall = recall_score(data.true, data.pred)
f1 = f1_score(data.true, data.pred)

show('accuracy', 'precision', 'recall', 'f1')
answer('3-3-2.txt', accuracy, precision, recall, f1)

accuracy: 0.535
precision: 0.558441558442
recall: 0.421568627451
f1: 0.480446927374


# 4
Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:
- для логистической регрессии — вероятность положительного класса (колонка score_logreg),
- для SVM — отступ от разделяющей поверхности (колонка score_svm),
- для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),
- для решающего дерева — доля положительных объектов в листе (колонка score_tree).

Загрузите этот файл.

In [42]:
scores = pd.read_csv('scores.csv')

print(scores.keys())
print(scores[:5])

Index(['true', 'score_logreg', 'score_svm', 'score_knn', 'score_tree'], dtype='object')
   true  score_logreg  score_svm  score_knn  score_tree
0     0      0.683832   0.145976   0.787063    0.500000
1     1      0.801966   0.239511   1.000000    0.833333
2     0      0.382315  -0.245701   0.000000    0.000000
3     1      0.506797  -0.137058   0.000000    0.105263
4     1      0.488781  -0.154148   0.000000    0.105263


# 5
Посчитайте площадь под ROC-кривой для каждого классификатора. Какой классификатор имеет наибольшее значение метрики AUC-ROC (укажите название столбца)? Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

In [44]:
def compute_roc_auc_score(y_score):
    return roc_auc_score(y_true=scores.true, y_score=y_score)

In [50]:
auc = {score: roc_auc_score(scores.true, scores[score]) for score in scores.keys()[1:]}
auc_max = max(auc, key=auc.get)
print('Scores: ', auc, '\nMax: ', auc_max, ' ', auc[auc_max])

answer('3-3-3.txt', auc_max)

Scores:  {'score_knn': 0.63515406162464982, 'score_tree': 0.69192677070828335, 'score_logreg': 0.71918767507002801, 'score_svm': 0.70868347338935567} 
Max:  score_logreg   0.71918767507


# 6
Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ?

Чтобы получить ответ на этот вопрос, найдите все точки precision-recall-кривой с помощью функции sklearn.metrics.precision_recall_curve. Она возвращает три массива: precision, recall, thresholds. В них записаны точность и полнота при определенных порогах, указанных в массиве thresholds. Найдите максимальной значение точности среди тех записей, для которых полнота не меньше, чем 0.7.

Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.42. При необходимости округляйте дробную часть до двух знаков.

In [60]:
curve = {}

for score in scores.keys()[1:]:
    df = pd.DataFrame(columns=('precision', 'recall'))
    df.precision, df.recall, thresholds = precision_recall_curve(scores.true, scores[score])
    curve[score] = df[df['recall'] >= 0.7]['precision'].max()
    
print(curve)

best_model = max(curve, key=curve.get)
print(best_model)

answer('3-3-4.txt', best_model)

{'score_knn': 0.60655737704918034, 'score_tree': 0.6517857142857143, 'score_logreg': 0.63025210084033612, 'score_svm': 0.6228070175438597}
score_tree


Наилучшую точность при таком ограничении показывает решающее дерево.